In [1]:
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pandas as pd
import numpy as np
from tools.traj_utils import trace
from scipy.stats import ttest_ind as ttest

In [2]:
cwd = os.getcwd() # Current working directory


wd = Path(cwd).parents[0] # project working directory

In [3]:
dataDir = Path(os.path.join(wd, 'RAW_EXPERIMENT_DATA', 'TWO-HUMAN_HAs')) # Directory of all human-human sessions
policy =  '/CollinearAngle' #Policy to be evaluated, can be one of '/CollinearAngle', '/CollinearDistance', '/Angle', '/Distance', '/ContainmentZone'
simDir= Path(wd, 'OtherResults', 'AA-AA_SimulationData' + policy)

In [4]:
first_trial = 7
last_trial = 24 + 1
numTrials = last_trial - first_trial
bin_size = 5
xlim = 60
ylim = 45
threshold = 10
numTicksX = 7 #for plots
numTicksY = 5
numPlayers = 21 #could automate this 
savefig = True
weighted_eval = True

In [ ]:
def do(player):
    savefig_dir = os.path.join(wd, 'Plots', "Experiment1_Player" + str(player))
    #if savefig_dir doesn't exist, create it
    if not os.path.exists(savefig_dir):
        os.makedirs(savefig_dir)
    tcount = 0
    traces_sim = np.zeros(last_trial-first_trial) -1
    traces_human = np.zeros((last_trial-first_trial, numPlayers)) -1
    for trial in range(first_trial,last_trial):
        X = np.array([])
        Z = np.array([])
        trial_ID = "{:02}".format(trial)
        filePaths = [path for path in dataDir.rglob('*trialIdentifier'+trial_ID+'*')]
        for filePath in filePaths:
            trialData = pd.read_csv(filePath)

            X = np.append(X, trialData['p%dx' % (player)].to_numpy())
            Z = np.append(Z, trialData['p%dz' % (player)].to_numpy())

        simFile = [path for path in simDir.rglob('*trialIdentifier'+trial_ID+'*')][0]
        simData = pd.read_csv(simFile)

        h, _, _ = np.histogram2d(x = X.flatten(), y = Z.flatten(),  bins = (int(120/bin_size), int(90/bin_size)), range = ((-xlim, xlim), (-ylim,ylim)))

        fig, _ = plt.subplots(1, 3, figsize=(16,4))
        axes = fig.get_axes()

        ax1 = axes[0]
        ax2 = axes[1]
        ax3 = axes[2]

        ax1.plot(X, Z, 'k.',  markersize = .05)
        ax1.plot(simData['hA%dx' % (player)], simData['hA%dz' % (player)], color = 'r' )
        ax1.set_xlim(left = -xlim, right = xlim)
        ax1.set_ylim(bottom=-ylim, top=ylim)
        ax1.set_xlabel(r'$x[\mathrm{m}]$')
        ax1.set_ylabel(r'$y[\mathrm{m}]$')
        ax1.plot(trialData.t0x[0], trialData.t0z[0], color = 'lime', marker = 'o')
        ax1.plot(trialData.t1x[0], trialData.t1z[0], color = 'lime', marker = 'o')
        ax1.plot(trialData.t2x[0], trialData.t2z[0], color = 'lime', marker = 'o')

        if 't3x' in trialData.columns:
            ax1.plot(trialData.t3x[0], trialData.t3z[0], color = 'lime', marker = 'o')
        if 't4x' in trialData.columns:   
            ax1.plot(trialData.t4x[0], trialData.t4z[0], color = 'lime', marker = 'o')

        ax1.plot(simData['hA%dx' % (player)][0], simData['hA%dz' % (player)][0], color='goldenrod', marker = 'o')
        ax1.set_xticks(ticks=(-60,-40,-20,0,20,40,60), labels = (-60,-40,-20,0,20,40,60))
        ax1.set_yticks(ticks = (-40, -20, 0, 20,40), labels = (-40, -20, 0, 20,40))
        ax1.set_aspect('equal')

        weighted_heatmap = np.sqrt((h.T[::-1]))
        ax2.matshow(weighted_heatmap)# ,aspect = 90/120)
        ax2.plot(simData['hA%dx' % (player)]*np.shape(h)[0]/(xlim*2) + np.shape(h)[0]/2 - 1/2, -simData['hA%dz' % (player)]*np.shape(h)[1]/(ylim*2) + np.shape(h)[1]/2 - 1/2,  color = 'r' )
        ax2.set_xticks(ticks = np.arange(numTicksX) * np.shape(h)[0] / (numTicksX -1) - .5, labels=(-60,-40,-20,0,20,40,60))
        ax2.set_yticks(ticks = np.arange(numTicksY) * 4 + 1 -.5 , labels=(40, 20, 0, -20,-40)) #hack to get right labels
        ax2.get_xaxis().set_visible(True)
        ax2.xaxis.tick_bottom()
        ax2.get_yaxis().set_visible(True)
        ax2.set_xlabel(r'$x[\mathrm{m}]$')
        ax2.set_ylabel(r'$y[\mathrm{m}]$')
        ax2.set_xlim(left = -xlim*np.shape(h)[0]/(xlim*2) + np.shape(h)[0]/2 - 1/2, right = xlim*np.shape(h)[0]/(xlim*2) + np.shape(h)[0]/2 - 1/2)
        ax2.set_ylim(bottom=ylim*np.shape(h)[1]/(ylim*2) + np.shape(h)[1]/2 - 1/2, top=-ylim*np.shape(h)[1]/(ylim*2) + np.shape(h)[1]/2 - 1/2)

        binary_heatmap = weighted_heatmap > threshold
        ax3.matshow(binary_heatmap, cmap='hot')# ,aspect = 90/120)
        ax3.plot(simData['hA%dx' % (player)]*np.shape(h)[0]/(xlim*2) + np.shape(h)[0]/2 -1/2 ,-simData['hA%dz' % (player)]*np.shape(h)[1]/(ylim*2) + np.shape(h)[1]/2 -1/2,  color = 'r' )
        ax3.set_xticks(ticks = np.arange(numTicksX) * np.shape(h)[0] / (numTicksX -1) - .5, labels=(-60,-40,-20,0,20,40,60))
        ax3.set_yticks(ticks = np.arange(numTicksY) * 4 + 1 -.5 , labels=(40, 20, 0, -20,-40)) #hack to get right labels
        ax3.get_xaxis().set_visible(True)
        ax3.xaxis.tick_bottom()
        ax3.get_yaxis().set_visible(True)
        ax3.set_xlabel(r'$x[\mathrm{m}]$')
        ax3.set_ylabel(r'$y[\mathrm{m}]$')
        ax3.set_xlim(left = -xlim*np.shape(h)[0]/(xlim*2) + np.shape(h)[0]/2 - 1/2, right = xlim*np.shape(h)[0]/(xlim*2) + np.shape(h)[0]/2 - 1/2)
        ax3.set_ylim(bottom=ylim*np.shape(h)[1]/(ylim*2) + np.shape(h)[1]/2 - 1/2, top=-ylim*np.shape(h)[1]/(ylim*2) + np.shape(h)[1]/2 - 1/2)



        metric = weighted_heatmap if weighted_eval else binary_heatmap


        traces_sim[tcount] =trace(metric, np.array([simData['hA%dx' % (player)].to_numpy(), simData['hA%dz' % (player)].to_numpy()]).T, bin_size = bin_size, xlim = xlim, ylim = ylim)

        hcount = 0
        for filePath in filePaths: # for each participant with the given trial ID
            trialData = pd.read_csv(filePath)
            traces_human[tcount][hcount] = trace(metric, np.array([trialData['p%dx' % (player)].to_numpy(), trialData['p%dz' % (player)].to_numpy()]).T, bin_size = bin_size, xlim = xlim, ylim = ylim)
            hcount += 1

        tcount += 1
        if savefig:
            plt.savefig(os.path.join(savefig_dir, 'Experiment1_Player' + str(player) + '_Trial' + str(trial) + '.png'))
            plt.close()
        plt.figure()
        
    columns = ["Trial", "Mean", "SD", "Sim"]
    table = pd.DataFrame(np.array([np.arange(first_trial, last_trial),np.mean(traces_human, axis = 1), np.std(traces_human, axis = 1), traces_sim]).T, columns = columns).astype({"Trial":"int"})
    meanRow =table.apply(['mean'])
    meanRow.Trial['mean'] = "Average"
    output = pd.concat([table, meanRow]).round(2)
    
    return output, table

In [6]:
output0, table0 = do(player = 0)
output1, table1 = do(player = 1)

C:\Users\MQ20208365\AppData\Local\Temp\ipykernel_22916\4217898249.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, _ = plt.subplots(1, 3, figsize=(16,4))
C:\Users\MQ20208365\AppData\Local\Temp\ipykernel_22916\4217898249.py:95: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>